In [1]:
import sys
from glob import glob

import h5py
import pygrib
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [2]:
sys.path.insert(0, '/glade/u/home/ksha/NCAR/')
sys.path.insert(0, '/glade/u/home/ksha/NCAR/libs/')

from namelist import *
import data_utils as du
#import preprocess_utils as pu

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
with h5py.File(save_dir+'HRRR_domain.hdf', 'r') as h5io:
    lon_3km = h5io['lon_3km'][...]
    lat_3km = h5io['lat_3km'][...]
    lon_80km = h5io['lon_80km'][...]
    lat_80km = h5io['lat_80km'][...]
    land_mask_80km = h5io['land_mask_80km'][...]
    land_mask_3km = h5io['land_mask_3km'][...]

In [5]:
# names = glob('/glade/campaign/mmm/parc/sobash/HRRRv4/2019100200/*')

In [6]:
lead = 2
HRRR_inds_p1 = [601, 614, 617, 621, 630, 671, 674, 679, 690, 708, 709, 736, 737, 742, 743]
HRRR_inds_p2 = [601, 614, 617, 621, 630, 671, 674, 679, 690, 708+4, 709+4, 736+4, 737+4, 742+4, 743+4]

In [7]:
# var_names = []

# with pygrib.open(names[0]) as grbio:
#     var_list = grbio()
#     for i, ind in enumerate(HRRR_inds_p1):
#         var_names.append(str(grbio[ind])[:35])

In [8]:
# ['Max/Comp Radar',
#  'MSLP',
#  'UH 2-5 km',
#  'UH 0-2 km',
#  'Graupel mass',
#  'T 2m',
#  'Dewpoint 2m',
#  'SPD 10m',
#  'APCP',
#  'CAPE',
#  'CIN',
#  'SRH 0-3 km',
#  'SRH 0-1 km',
#  'U shear 0-6 km',
#  'V shear 0-6 km']

In [9]:
base_ref = datetime(2019, 10, 1)
date_list_v4x = [base_ref + timedelta(days=day) for day in range(429)]

In [10]:
filepath = '/glade/campaign/mmm/parc/sobash/HRRRv4/{}/{}'

In [12]:
i = 1
var_names_p1 = []
date_temp = date_list_v4x[i]

name_folder = datetime.strftime(date_temp, '%Y%m%d00')
filename_pattern = '*0000{:02d}00'.format(lead)
filename = sorted(glob(filepath.format(name_folder, filename_pattern)))[0]

with pygrib.open(filename) as grbio:
    var_list = grbio()
    for i, ind in enumerate(HRRR_inds_p1):
        var_names_p1.append(str(grbio[ind])[:55])
        
i = 241
var_names_p2 = []
date_temp = date_list_v4x[i]

name_folder = datetime.strftime(date_temp, '%Y%m%d00')
filename_pattern = '*0000{:02d}00'.format(lead)
filename = sorted(glob(filepath.format(name_folder, filename_pattern)))[0]

with pygrib.open(filename) as grbio:
    var_list = grbio()
    for i, ind in enumerate(HRRR_inds_p2):
        var_names_p2.append(str(grbio[ind])[:35])

In [13]:
var_names_p1

['601:196:196 (instant):lambert:atmosphere:level 0 10:fcs',
 '614:198:198 (instant):lambert:meanSea:level 0:fcst time',
 '617:199:199 (max):lambert:heightAboveGroundLayer:levels',
 '621:199:199 (max):lambert:heightAboveGroundLayer:levels',
 '630:74:74 (max):lambert:unknown:level 0 200:fcst time 1',
 '671:2 metre temperature:K (instant):lambert:heightAbove',
 '674:2 metre dewpoint temperature:K (instant):lambert:he',
 '679:10 metre wind speed:m s**-1 (max):lambert:heightAbo',
 '690:Total precipitation:kg m-2 (accum):lambert:surface:',
 '708:Convective available potential energy:J kg-1 (insta',
 '709:Convective inhibition:J kg-1 (instant):lambert:surf',
 '736:Storm relative helicity:J kg**-1 (instant):lambert:',
 '737:Storm relative helicity:J kg**-1 (instant):lambert:',
 '742:Vertical u-component shear:s**-1 (instant):lambert:',
 '743:Vertical v-component shear:s**-1 (instant):lambert:']

In [17]:
var_names_p2

['601:196:196 (instant):lambert:atmos',
 '614:198:198 (instant):lambert:meanS',
 '617:199:199 (max):lambert:heightAbo',
 '621:199:199 (max):lambert:heightAbo',
 '630:74:74 (max):lambert:unknown:lev',
 '671:2 metre temperature:K (instant)',
 '674:2 metre dewpoint temperature:K ',
 '679:10 metre wind speed:m s**-1 (ma',
 '690:Total precipitation:kg m-2 (acc',
 '712:Convective available potential ',
 '713:Convective inhibition:J kg-1 (i',
 '740:Storm relative helicity:J kg**-',
 '741:Storm relative helicity:J kg**-',
 '746:Vertical u-component shear:s**-',
 '747:Vertical v-component shear:s**-']

In [15]:
prefix = 'v4'

In [14]:
# index change: 2020,5,29,0,0,0

In [16]:
date_list_v4x[241]

datetime.datetime(2020, 5, 29, 0, 0)

In [22]:
# Max number of possible files
L = len(date_list_v4x)

# Allocation (one array for one lead time)
VARs = np.empty((L, 1059, 1799, 23))


for i in range(240, 243, 1):

    var_names_temp = []
    date_temp = date_list_v4x[i]
    print(date_temp)
    
    name_folder = datetime.strftime(date_temp, '%Y%m%d00')
    filename_pattern = '*0000{:02d}00'.format(lead)
    filename = sorted(glob(filepath.format(name_folder, filename_pattern)))

    if len(filename) > 0:
        filename = filename[0]
    
        if i >= 241:
            with pygrib.open(filename) as grbio:
                var_list = grbio()
                for j, ind in enumerate(HRRR_inds_p2):
                    VARs[i, ..., j] = grbio[ind].values
                    var_names_temp.append(str(grbio[ind])[:35])
                    
            if var_names_temp != var_names_p2:
                print('var names mismatch')
                VARs[i, ...] = np.nan
                
        else:
            with pygrib.open(filename) as grbio:
                var_list = grbio()
                for j, ind in enumerate(HRRR_inds_p1):
                    VARs[i, ..., j] = grbio[ind].values
                    var_names_temp.append(str(grbio[ind])[:35])
                    
            if var_names_temp != var_names_p1:
                print('var names mismatch')
                VARs[i, ...] = np.nan
    else:
        print(filepath.format(name_folder, filename_pattern)+' is missing')
        VARs[i, ...] = np.nan

In [25]:
var_names_temp == var_names_p2

True